In [3]:
# If cloning is needed -- i.e., if hangman.py and word counts are not already
# available in file, you can run this
!git clone https://github.com/ucsd-cse150a-w25/hw1.git
!cp hw1/hangman.py hangman.py
!cp hw1/hw1_word_counts_05.txt hw1_word_counts_05.txt

fatal: destination path 'hw1' already exists and is not an empty directory.


In [4]:
from hangman import hangman_game

# Play the game by yourself
hangman_game()



  +---+
  O   |
 /|\  |
 / \  |
     ===

Word: _ _ _ C _
Tried letters: A, B, C, D, E, F, G
Game over! The word was:  WHICH


0

In [5]:
import random

def random_inference(
    letters_tried: set[str],
    word_pattern: list[str],
    word_counts: dict[str, int]
) -> str:
    '''
    Random inference for playing hangman. This should be a simple method which returns a letter
    that is NOT in letters_tried but any other letter, at random.

    Hint: use the random.choice method
    '''
    # TODO: Implement random inference code to guess hangman
    alphabet = set("ABCDEFGHIJKLMNOPQRSTUVWXYZ")
    remaining_letters = list(alphabet - letters_tried)
    return random.choice(remaining_letters) if remaining_letters else ''

hangman_game(random_inference)



  +---+
  O   |
 /|\  |
 / \  |
     ===

Word: O _ _ _ _
Tried letters: C, F, O, R, U, X, Y
Game over! The word was:  OPENS


0

In [8]:
import collections
import numpy as np

def bayesian_inference(
    letters_tried: set[str],
    word_pattern: list[str],
    word_counts: dict[str, int]
) -> str:
    '''
    Bayesian inference method for playing hangman. The parameters given are as follows:

    - letters_tried (set[str]): A set of strings which consist of all the letters that have already
        been tried. For example, if 'A', 'E' has been guessed, `letters_tried = {'A', 'E'}`
    - word_pattern (list[str]): A list of single characters that describe the current guess state.
        For example, if the hangman state is _AB__, `word_pattern = ['_', 'A', 'B', '_', '_']`
    - word_counts (dict[str, int]): The word counts dictionary which contains all possible 5 letter
        words in our hangman game and their respective counts. For example, a key value pair could
        be 'AARON': 413.

    Return type: a single character string as your next best guess.
    '''
    # TODO: Implement inference code to play hangman optimally
    valid_words = {
        word: count
        for word, count in word_counts.items()
        if all((word[i] == word_pattern[i] or word_pattern[i] == '_') for i in range(5))
        and all(letter not in word for letter in letters_tried if letter not in word_pattern)
    }

    total_valid_count = sum(valid_words.values())
    posterior_probs = {
        word: count / total_valid_count for word, count in valid_words.items()
    }

    letter_probs = collections.defaultdict(float)

    for word, prob in posterior_probs.items():
        for i, letter in enumerate(word):
            if word_pattern[i] == '_' and letter not in letters_tried:
                letter_probs[letter] += prob

    if letter_probs:
        return max(letter_probs, key=letter_probs.get)
    else:
        return np.random.choice([l for l in "ABCDEFGHIJKLMNOPQRSTUVWXYZ" if l not in letters_tried])


# Run the game
hangman_game(bayesian_inference)



  +---+
  O   |
      |
      |
     ===

Word: A B O U T
Tried letters: A, B, E, O, T, U
Congratulations! You guessed the word:  ABOUT


1

In [10]:
from typing import Optional, Callable
# from tqdm import tqdm

def benchmark(
    inference: Optional[Callable],
    num_runs: int = 1000,
    seed: int = 0
) -> None:
    '''
    Benchmark method for testing out the bayesian inference method. The parameters given are:

    - inference: The function that should match the bayesian_inference() method above.
    - seed: The seed to pass into the hangman_game function.

    Return type: none.

    Print the accuracy out of num_runs. If the function throws an error, it should count as a fail.
    '''
    # TODO: Implement benchmark testing for a given function
    # Hint: use `random.seed(seed)` ONCE at the start
    # Hint: pass `interactive = False` into the hangman_game function to run faster without outputs.
    # Optional: use the `tqdm` module to keep track of the evaluation progress.

    random.seed(seed)
    successes = 0

    for _ in range(num_runs):
        try:
            result = hangman_game(inference, interactive=False)
            if result:
                successes += 1
        except Exception:
            pass

    accuracy = (successes / num_runs) * 100
    print(f"{inference.__name__} Accuracy: {accuracy:.2f}% over {num_runs} runs")


benchmark(random_inference)
benchmark(bayesian_inference)

Accuracy: 0.40% over 1000 runs
Accuracy: 93.00% over 1000 runs
